In [ ]:
#| default_exp core

In [ ]:
%load_ext autoreload
%autoreload 2

# pipeline

In [ ]:
from ergativegpt.src import *

In [ ]:
import pandas as pd

from langchain.chains.openai_functions import create_structured_output_runnable
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from tqdm.notebook import tqdm

from sklearn.metrics import precision_recall_fscore_support,  accuracy_score

import altair as alt

from datetime import datetime

## load data

In [ ]:
df = pd.read_excel('../in/Data_GPT.xlsx')
df

,Token,Corpus,Source,Year,Variety,verbRealization,verbLemma,Transitivity,Transitivity_GPT,subjectAnimacy,subjectAnimacy_GPT,subjectRole,subjectRole_GPT,Construction,Construction_GPT
0,Android 12 will let users play games as they'r...,NOW,https://www.digitaltrends.com/gaming/android-1...,2021,AmE,downloading,download,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
1,There's also a new feature that will let you u...,NOW,https://www.androidheadlines.com/2021/10/googl...,2021,AmE,downloading,download,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
2,Click Next and Windows 10 will install.,NOW,https://uk.pcmag.com/migrated-3765-windows-10/...,2022,AmE,install,install,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
3,"If your Kindle is connected to Wi-Fi, the upda...",NOW,https://techcrunch.com/2021/09/13/kindle-relea...,2021,AmE,install,install,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
4,"The first two are available right now, while W...",NOW,https://techcrunch.com/2022/06/30/raspberry-pi...,2022,AmE,shipping,ship,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,He's been uploading videos so students can sta...,NOW,https://www.thenation.com/article/society/arka...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
96,"Once you've been uploading content regularly, ...",NOW,https://www.entrepreneur.com/article/381562,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
97,The cast members have also been uploading phot...,NOW,https://www.soompi.com/article/1491029wpp/lee-...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
98,The parody videos that' Squid Game' viewers ha...,NOW,https://www.soompi.com/article/1491029wpp/lee-...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN


## set up model

In [ ]:
model="gpt-3.5-turbo"
# model="gpt-4"

llm = ChatOpenAI(model=model, temperature=0)

In [ ]:
with open('../in/prompt.md', 'r') as f:
        prompt_txt = f.read()

In [ ]:
prompt = ChatPromptTemplate.from_messages([
        ("system", prompt_txt),
        ("human", 'Please classify the following sentence: {input}')
])

In [ ]:
runnable = create_structured_output_runnable(UtteranceClassification, llm, prompt)

## run processing

In [ ]:
#| notest

results_list = []

df_dev = (df
    .sample(10)
)

for _, row in tqdm(df_dev.iterrows(), total=len(df_dev)):
    try:
        result = runnable.invoke({"input": row['verbRealization'] + ' in: ' + row['Token']})
        row_data = {
            'text': row['Token'],
            'transitivity': row['Transitivity'],
            'gpt_transitivity': result.gpt_transitivity,
            'causativity': row['Construction'],
            'gpt_causativity': result.gpt_causativity,
            'subject_animacy': row['subjectAnimacy'],
            'gpt_subject_animacy': result.gpt_subject_animacy,
            'subject_role': row['subjectRole'],
            'gpt_subject_role': result.gpt_subject_role,

            'gpt_subject': result.gpt_subject,
            'gpt_verb': result.gpt_verb,
            'gpt_object': result.gpt_object
        }
        results_list.append(row_data)
    except:
        continue

results = pd.DataFrame(results_list)

In [ ]:
if 'results' not in globals():
    results = pd.read_csv('../out/2023-12-14_23:27_results.csv')

## calculate metrics

align labels of factor levels between human and gpt labelling

In [ ]:
results['transitivity'] = results['transitivity'].replace({'Transitive': 'transitive', 'Intransitive': 'intransitive'})
results['causativity'] = results['causativity'].replace({0: 'causative', 1: 'anticausative'})
results['subject_animacy'] = results['subject_animacy'].replace({'Animate': 'animate', 'Inanimate': 'inanimate'})
results['subject_role'] = results['subject_role'].replace({'Agent': 'agent', 'Patient': 'patient'})

set variables and positive labels

In [ ]:
vars = {'transitivity': 'intransitive',
	'causativity': 'anticausative',
	'subject_animacy': 'inanimate',
	'subject_role': 'patient'}

calculate metrics

In [ ]:
metrics_vars = []
for var in vars.items():
	metrics_var = get_metrics(results, var[0], var[1])
	metrics_vars.append(metrics_var)

metrics = pd.concat(metrics_vars)

metrics

,variable,metric,score
0,transitivity,precision,0.98
1,transitivity,recall,0.78
2,transitivity,accuracy,0.85
3,transitivity,F1,0.87
0,causativity,precision,0.94
1,causativity,recall,0.34
2,causativity,accuracy,0.66
3,causativity,F1,0.50
0,subject_animacy,precision,1.00
1,subject_animacy,recall,0.93


plot metrics

In [ ]:
chart = alt.Chart(metrics).mark_bar().encode(
	y='score:Q',
	x=alt.X('metric:N', sort=metrics['metric'].tolist()),
	color=alt.Color('metric', legend=None),
).facet(column='variable:N')

chart

alt.FacetChart(...)

## save results